In [1]:
import sys
%conda install --yes --prefix {sys.prefix} gspread oauth2client

Channels:
 - conda-forge
Platform: linux-64
Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [2]:
import datetime
import pandas as pd
import numpy as np

def months_between(d1, d2):
    return d1.month - d2.month + 12*(d1.year - d2.year)

# Calculate the weight based on linear decay
def linear_decay_weight(date, current_date=datetime.datetime.now()):
  age_in_months = months_between(current_date, date)
  weight = max(1 - (age_in_months / 13), 0)
  return weight

# Weight function 1: Exponential decay
def exponential_decay_weight(date, current_date=datetime.datetime.now()):
  age_in_months = months_between(current_date, date)
  weight = np.exp(-age_in_months / 13)
  return weight

# Weight function 2: Quadratic decay
def quadratic_decay_weight(date, current_date=datetime.datetime.now()):
  age_in_months = months_between(current_date, date)
  weight = max(1 - (age_in_months / 13) ** 2, 0)
  return weight

## Daten laden und vorbereiten

In [39]:
# Group transactions by month and sum up their amount
df = pd.read_csv('transactions.csv')
df['month'] = pd.to_datetime(df['transaction date']).dt.to_period('M').dt.to_timestamp('M')
df['month'] = df['month'] - pd.offsets.MonthBegin(1)
df_grouped = df.groupby(['month', 'category'])[['amount']].sum().reset_index()

# Sort the dataframe by month in ascending order
df_sorted = df_grouped.sort_values('month')

## Untersuchen, welche Gewichtsfunktion am besten geeignet ist

In [ ]:
# Generate subsets of the dataframe
subsets = []
months = df_sorted['month'].unique()
for subsets_upper_limit in range(7, len(months)):
  subset = df_sorted[df_sorted['month'] < months[subsets_upper_limit]]
  solution = df_sorted[df_sorted['month'] == months[subsets_upper_limit]]
  subsets.append([subset[:-1], solution])

stats = {}
for category in df['category'].unique():
  stats[category] = {'month': [], 'average': [], 'linear_decay': [], 'exponential_decay': [], 'quadratic_decay': []}

for [subset, solutions] in subsets:
  current_date = subset['month'].max() + pd.DateOffset(months=1)
  test_data = subset.copy()
  test_data['linear_decay_weigths'] = test_data.apply(lambda row: linear_decay_weight(row['month'], current_date=current_date), axis=1)
  test_data['exponential_decay_weigths'] = test_data.apply(lambda row: exponential_decay_weight(row['month'], current_date=current_date), axis=1)
  test_data['quadratic_decay_weigths'] = test_data.apply(lambda row: quadratic_decay_weight(row['month'], current_date=current_date), axis=1)
  for category in test_data['category'].unique():
    solution = solutions[solutions['category'] == category]['amount']
    if solution.empty:
      continue
    solution = solution.values[0]
    category_test_data = test_data[test_data['category'] == category]
    average = category_test_data['amount'].mean()
    linear_decay = (category_test_data['amount'] * category_test_data['linear_decay_weigths']).sum() / category_test_data['linear_decay_weigths'].sum()
    exponential_decay = (category_test_data['amount'] * category_test_data['exponential_decay_weigths']).sum() / category_test_data['exponential_decay_weigths'].sum()
    quadratic_decay = (category_test_data['amount'] * category_test_data['quadratic_decay_weigths']).sum() / category_test_data['quadratic_decay_weigths'].sum()
    stats[category]['month'].append(current_date)
    stats[category]['average'].append(solution - average)
    stats[category]['linear_decay'].append(solution - linear_decay)
    stats[category]['exponential_decay'].append(solution - exponential_decay)
    stats[category]['quadratic_decay'].append(solution - quadratic_decay)

import matplotlib.pyplot as plt

for category in stats.keys():
  data = stats[category]
  fig, axs = plt.subplots(2, 2, figsize=(10, 8), sharey=True)
  for (variant, (axis_x, axis_y)) in zip(['average', 'linear_decay', 'exponential_decay', 'quadratic_decay'], [[0,0], [0,1], [1,0], [1,1]]):
    if len(data[variant]) == 0:
      continue
    print(data[variant])
    avg = np.median(data[variant])
    p25 = np.percentile(data[variant], 25)
    p75 = np.percentile(data[variant], 75)

    axs[axis_x, axis_y].hist(data[variant], color='green', ec='black', bins=25)
    axs[axis_x, axis_y].set_title(f'{variant}, P25: {p25:.2f}, Median: {avg:.2f}, P75: {p75:.2f}')
  fig.suptitle(category)
  plt.tight_layout()
  plt.show()

In [40]:
current_date = datetime.datetime.now()
df_sorted['linear_decay_weigths'] = df_sorted.apply(lambda row: linear_decay_weight(row['month'], current_date=current_date), axis=1)
category_stats = [["category", "average", "linear_decay"]]
for category in df_sorted['category'].unique():
  category_data = df_sorted[df_sorted['category'] == category]
  if category_data['linear_decay_weigths'].sum() == 0:
    linear_decay = np.float32(0)
  else:
    linear_decay = (category_data['amount'] * category_data['linear_decay_weigths']).sum() / category_data['linear_decay_weigths'].sum()
  average = category_data['amount'].mean()
  category_stats.append([category, average.item(), linear_decay.item()])

In [41]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# Set up credentials
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('credentials.json', scope)
client = gspread.authorize(credentials)

sheet = client.open('Finanzplanung / Bospar')
worksheet = sheet.worksheet("Category Stats")
print(category_stats)
worksheet.update(category_stats)
print("Statistics posted to Google Sheets successfully!")

[['category', 'average', 'linear_decay'], ['Benzin', -1603.2501785714287, -1765.3158241758242], ['Parken unterwegs', -296.01759259259256, -379.34487804878046], ['undefined', 66784.71327272727, 33801.32758241759], ['non-food Verbrauchsgegenst√§nde', -786.2229166666666, -1372.110641025641], ['Urlaub: Unterk√ºnfte', -8233.736551724138, -19064.330652173918], ['Urlaub: Autokosten (ohne Sprit)', -3115.245714285714, -19350.611249999994], ['Tierarzt & Medikamente (#commonspar)', -1540.6237037037038, -1693.5768571428573], ['Starting Balances', 74778.74187499999, 871794.2600000001], ['Rent', -7864.2, 0.0], ['Parken daheim [426/m]', -403.52272727272725, -436.8], ['Medikamente', -392.09820512820517, -481.73878787878795], ['Project 4 Walls', -506.70799999999997, 0.0], ['Income', 95178.9314814815, 98567.54153846153], ['Ausw√§rts Essen', -1750.3630769230767, -1840.9552564102567], ['Kaffe oder Fika', -479.59959183673465, -763.7546153846155], ['Dog Walker', -2410.0, 0.0], ['Electric', -391.25, 0.0], ['

/tmp/ipykernel_17424/4215209258.py:12: DeprecationWarning: [Deprecated][in version 6.0.0]: Method signature's arguments 'range_name' and 'values' will change their order. We recommend using named arguments for minimal impact. In addition, the argument 'values' will be mandatory of type: 'List[List]'. (ex) Worksheet.update(values = [[]], range_name=) 
  worksheet.update(category_stats)


Statistics posted to Google Sheets successfully!
